In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import plotnine as pn


## 1. The data

For this lesson, we'll work with a new dataset of unknown origin (for the moment).  Let's load the data and have a look.  Our goal is to build a model that can predict $y$.

In [ ]:
#df = pd.read_csv('regularization.csv')
df = pd.read_csv('/blue/zoo4926/share/Jupyter_Content/data/regularization.csv')

In [ ]:
df

What should we do with this? We have 20 $x$ variables and 50 observations of those variables. How can we build a good predictive model for these data?

## 2. Multiple linear regression

Let's explore multiple linear regression first to see what happens. _Note that, in the interest of time, I am leaving out exploratory data analyses and visualizations, which is where you should start!_

In [ ]:
x = df.drop(columns=['y'])
y = df.y

model = LinearRegression()
model.fit(x, y)

def mse(model, x, y):
    y_hat = model.predict(x)
    err = np.mean((y - y_hat)**2)
    return err

print('R^2:', model.score(x, y), 'MSE:', mse(model, x, y))

Now that we've got a model, let's use 5-fold cross-validation to estimate how well it will work for new (i.e., non-training) data.

In [ ]:
kf = KFold(n_splits=5)

res = cross_val_score(model, x, y, cv=kf, scoring=mse)
cv_mse = res.mean()
res = cross_val_score(model, x, y, cv=kf)
cv_r2 = res.mean()
print('c-v R^2:', cv_r2, 'c-v MSE:', cv_mse)

## 3. Lasso regularization

Okay, so let's see what the lasso ($\ell_1$-regularized regression) does with these data.  We can use scikit-learn's `Lasso` estimator for this.  Note that scikit-learn uses `alpha` for the name of the parameter referred to as $\lambda$ virtually everywhere else (perhaps because `lambda` is a reserved word in Python).

As a general practice, you should always make sure that your predictor variables are on the same scale when fitting a regularized regression model.  We can have `Lasso` do this for us automatically by passing `normalize=True` when we create the `Lasso` object.

In [ ]:
model = Lasso(alpha=0.001, normalize=True)
model.fit(x, y)
print(model.coef_)

res = cross_val_score(model, x, y, cv=kf, scoring=mse)
cv_mse = res.mean()
res = cross_val_score(model, x, y, cv=kf)
cv_r2 = res.mean()
print('c-v R^2:', cv_r2, 'c-v MSE:', cv_mse)

## 4. Ridge regression

Now let's see what happens if we use ridge regression ($\ell_2$-regularized regression) with these data.  We can use scikit-learn's `Ridge` estimator to do this, and again, don't forget to make sure that your predictor variables are on the same scale.

In [ ]:
model = Ridge(alpha=1000.0, normalize=True)
model.fit(x, y)
print(model.coef_)

res = cross_val_score(model, x, y, cv=kf, scoring=mse)
cv_mse = res.mean()
res = cross_val_score(model, x, y, cv=kf)
cv_r2 = res.mean()
print('c-v R^2:', cv_r2, 'c-v MSE:', cv_mse)

## 5. How does $\lambda$ influence the parameter estimates?

Before we move on, let's take a closer look at how changing lambda influences the parameter estimates for both the lasso and ridge regression.

## 6. Choosing $\lambda$ and a modeling procedure

How should we choose the "best" value of $\lambda$ for the lasso or ridge regression?  How should we decide whether to use the lasso or ridge regression?  What ideas do you have?

In [ ]:
model = LassoCV(normalize=True)
model.fit(x, y)
print(model.coef_)
print(model.alpha_)

res = cross_val_score(model, x, y, cv=kf, scoring=mse)
cv_mse = res.mean()
res = cross_val_score(model, x, y, cv=kf)
cv_r2 = res.mean()
print('c-v R^2:', cv_r2, 'c-v MSE:', cv_mse)

In [ ]:
lambdas = np.geomspace(0.001, 10, 100)
model = RidgeCV(alphas=lambdas, normalize=True)
model.fit(x, y)
print(model.coef_)
print(model.alpha_)

res = cross_val_score(model, x, y, cv=kf, scoring=mse)
cv_mse = res.mean()
res = cross_val_score(model, x, y, cv=kf)
cv_r2 = res.mean()
print('c-v R^2:', cv_r2, 'c-v MSE:', cv_mse)

## 7. Why/how do these methods work? Is one better than the other?

We'll start by revealing where the data we've been modeling actually came from, which will give us some insight into how these methods work.  Then we'll dig a little deeper into how these methods are able to decrease the expected test error.

Key points to remember:
* The lasso can drive coefficients to 0; thus, i can perform _feature selection_.
* Ridge regression constrains coefficient values (_shrinks_ them), but does not remove them from the model.

Which is best depends on your data and the goals of the analysis.

## 8. Other methods

There are many other methods available for model/feature selection.  If you've had a statistics course, you've no doubt learned some of them.

Unfortunately, we don't have time to cover any of these alternative methods in detail in this course.  I will mention, though, that _dimensionality reduction_ is another widely used, general approach to dealing with model complexity and certainly worth learning about.  _Principle components regression_ is one example of this approach.

Another method I should mention is the _elastic net_, which combines $\ell_1$ and $\ell_2$ penalties into a single fitting procedure and can benefit from the desirable properties of both the lasso and ridge regression.

## 9. Data analysis lab

Now, put together everything that you've learned over the last few units (prediction error, bias and variance; methods for estimating test/prediction error; regularization techniques) and apply it to a data analysis problem.  Returning to the fuel efficiency dataset, your goal is to create a machine learning model that can accurately predict the fuel efficiency of a vehicle.

In [ ]:
#df = pd.read_csv('data/auto_mpg.csv')
df = pd.read_csv('/blue/zoo4926/share/Jupyter_Content/data/auto_mpg.csv')

In [ ]:
df
x = df.drop(columns=['mpg', 'name'])
y = df.mpg
x

In [ ]:
def mse(model, x, y):
    y_hat = model.predict(x)
    err = np.mean((y - y_hat)**2)
    return err


In [ ]:
kf = KFold(n_splits=10, shuffle=True)

model = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('lasso', LassoCV(normalize=True, fit_intercept=False))
])
model.fit(x, y)
print(model['lasso'].coef_)
print(model['lasso'].alpha_)

res = cross_val_score(model, x, y, cv=kf, scoring=mse)
cv_mse = res.mean()
res = cross_val_score(model, x, y, cv=kf)
cv_r2 = res.mean()
print('c-v R^2:', cv_r2, 'c-v MSE:', cv_mse)

In [ ]:
model['poly'].get_feature_names()
not_zero = model['lasso'].coef_ != 0
np.array(model['poly'].get_feature_names())[not_zero]

In [ ]:
kf = KFold(n_splits=10, shuffle=True)

lambdas = np.geomspace(0.01, 100)

model = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('ridge', RidgeCV(alphas=lambdas, normalize=True, fit_intercept=False))
])
model.fit(x, y)
print(model['ridge'].coef_)
print(model['ridge'].alpha_)

res = cross_val_score(model, x, y, cv=kf, scoring=mse)
cv_mse = res.mean()
res = cross_val_score(model, x, y, cv=kf)
cv_r2 = res.mean()
print('c-v R^2:', cv_r2, 'c-v MSE:', cv_mse)